https://www.acmicpc.net/step/26

## 최단경로
방향그래프가 주어지면 주어진 시작점에서 다른 모든 정점으로의 최단 경로를 구하는 프로그램을 작성하시오. (모든 간선의 가중치는 10 이하의 자연수)

In [2]:
import sys
from heapq import heappush, heappop
inf = 100000000
v, e = map(int, input().split())
k = int(input())
s = [[] for _ in range(v + 1)]
dp = [inf] * (v + 1)
heap = []

ValueError: invalid literal for int() with base 10: '1 2 3'

In [ ]:
def dijkstra(start):
    dp[start] = 0
    heappush(heap, [0, start])
    while heap:
        w, n = heappop(heap)
        for n_n, wei in s[n]:
            n_w = wei + w
            if n_w < dp[n_n]:
                dp[n_n] = n_w
                heappush(heap, [n_w, n_n])
for i in range(e):
    u, v, w = map(int, sys.stdin.readline().split())
    s[u].append([v, w])
dijkstra(k)
for i in dp[1:]:
    print(i if i != inf else "INF")